In [ ]:
from py_factorio_blueprints.util import *
from PIL import Image, ImagePalette
from collections import defaultdict


In [ ]:
def get_avg_color(img):
    sr, sg, sb, sa = 0, 0, 0, 0
    # print([x for x in img.getdata()])
    for r, g, b, a, in img.getdata():
        sr += r * a;
        sg += g * a
        sb += b * a
        sa += a
    return sr // sa, sg // sa, sb // sa

def color_dist(c1, c2):
    return (c1[0] - c2[0]) ** 2 + (c1[1] - c2[1]) ** 2 + (c1[2] - c2[2]) ** 2


In [ ]:
PATH = '/mnt/c/Program Files (x86)/Steam/steamapps/common/Factorio/data/base/graphics/icons'

MATERIALS = [
    'iron-plate',
    'copper-plate',
    'plastic-bar',
    'assembling-machine-1',
    'assembling-machine-2',
    'blueprint',
    'cliff-explosives',
    'coal',
    'concrete',
    'deconstruction-planner',
    'effectivity-module',
    'electronic-circuit',
    'explosives',
    'hazard-concrete',
    'iron-chest',
    'iron-ore',
    'landfill',
    'processing-unit',
    'productivity-module',
    'small-lamp',
    'solid-fuel',
    'speed-module',
    'stone',
    'steel-plate',
    'steel-chest',
    'stone-brick',
    'upgrade-planner',
    'wooden-chest'
]
COLORS = {}

for name in MATERIALS:
    img = Image.open(f"{PATH}/{name}.png").crop((0, 0, 64, 64)).convert("RGBA")
    COLORS[get_avg_color(img)] = name

COLORS


In [ ]:
def convert(image: Image, palette: list, W_ROUND=4, H=32):
    width, height = image.size
    width, height = (H * width) // (height * W_ROUND) * W_ROUND, H
    image = image.resize((width, height))

    data = [[None] * H for _ in range(width)]

    for x in range(width):
        for y in range(height):
            c_orig = image.getpixel((x, y))
            _, rgb = min((color_dist(c_orig, color), color) for color in palette)

            image.putpixel((x, y), rgb)
            data[x][y] = rgb

    return image, data


In [ ]:
cat = Image.open('cat.jpg')
cat, mp = convert(cat, COLORS.keys())
# cat.resize((cat.width * 8, cat.height * 8))


In [ ]:
cat = Image.open('cat.jpg')
cat.thumbnail((10**9, 32))

cat.resize((cat.width * 8, cat.height * 8))


In [ ]:
from futil import *
import futil
import importlib
importlib.reload(futil)


bl = Blueprint()

bl.create_constant(0, 0)
bl.create_decider(1, 0, 15)

bl.add_connection(0, 0, Blueprint.CONN_CONSTANT, 1, 0, Blueprint.CONN_DECIDER_IN, 'green')
bl.add_connection(1, 0, Blueprint.CONN_DECIDER_OUT, 1, 0, Blueprint.CONN_DECIDER_IN, 'green')

bl.pr()


In [ ]:
def generate_blueprint(data: list[list], palette: dict):
    W = len(data)
    assert W % 4 == 0 and all(len(col) == 32 for col in data)

    bl = Blueprint()

    for x, column in enumerate(data):
        X = x // 4
        C = [0 + 4 * (x % 4), 1 + 4 * (x % 4)]
        D = 2 + 4 * (x % 4)

        bl.create_constant(X, C[0])
        bl.create_constant(X, C[1])
        bl.create_decider (X, D, f_value=X)

        bl.add_connection(X, D, Blueprint.CONN_DECIDER_IN, X-1, D, Blueprint.CONN_DECIDER_IN, 'green', strict=False)
        bl.add_connection(X, D, Blueprint.CONN_DECIDER_IN, X, D-4, Blueprint.CONN_DECIDER_IN, 'green', strict=False)
        bl.add_connection(X, D, Blueprint.CONN_DECIDER_OUT, X-1, D, Blueprint.CONN_DECIDER_OUT, 'red', strict=False)

        bl.add_connection(X, C[0], Blueprint.CONN_CONSTANT, X, C[1], Blueprint.CONN_CONSTANT, 'red')
        bl.add_connection(X, C[1], Blueprint.CONN_CONSTANT, X, D, Blueprint.CONN_DECIDER_IN, 'red')

        for y, rgb in enumerate(column):
            item = palette[rgb]
            bl.add_item(X, C[y % 2], item, 1 << y)

    return bl


In [ ]:
Blueprint(string='0eNqtkdsKwjAQRP9lnqPQi7bmV0Skl0UW7KYkqVhK/t2kfREEQfAlZMLsmWGzoL1PNFoWD72AOyMO+rzA8U2ae3rz80jQYE8DFKQZkko+34jfdWZoWRpvLIICS09P6CxcFEg8e6YNt4r5KtPQko2GryCF0bg4ayTlJ97xsD8ozNBlvMScni11myFXiSGbdGkgS4el/j2Yo8rDJYSgPsrkv5Yp/1AmW8vENa171W/foPAg6zZcnZXVKa+quqiL4hjCC8Brj/o=')._root


In [ ]:
generate_blueprint(mp, COLORS)
